# Patch Prediction Models

<a href="https://colab.research.google.com/github/TIA-Lab/tiatoolbox/blob/master/examples/example_patchprediction.ipynb" target="_blank"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://kaggle.com/kernels/welcome?src=https://github.com/TIA-Lab/tiatoolbox/blob/master/examples/example_patchprediction.ipynb" target="_blank"><img src="https://kaggle.com/static/images/open-in-kaggle.svg" alt="Open In Colab"/></a>

_Please note that if you are selecting Kaggle badge, in the Kaggle notebook you have to login and enable the internet connection from the setting panel on the right in order to install tiatoolbox from the internet._

## About this notebook


Before running the notebook outside Colab, set up your Python environment, as explained in the 
[README](https://github.com/TIA-Lab/tiatoolbox/blob/master/README.md#install-python-package) file.

## Welcome to Tiatoolbox

In this example we will show how to use tiatoolbox for patch-level prediction using a range of deep learning models. Tiatoolbox can be used to make predictions on pre-extracted image patches or on larger image tiles / whole-slide images (WSIs), where image patches are extracted on the fly. WSI Patch-level predictions can subsequently be aggregated to obtain a segmentation map. In particular, we will introduce the use of our module
`patch_predictor` ([details](https://github.com/TIA-Lab/tiatoolbox/blob/master/tiatoolbox/models/classification/patchextraction.py)).

### First cell in bash
This cell prepares the Colab environment for the use of `tiatoolbox`. This cell produces no output if it has been run before in the same Colab session.

In [1]:
!apt-get -y install libopenjp2-7-dev libopenjp2-tools openslide-tools
!pip install tiatoolbox

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


## Importing related libraries
We will start by importing some libraries required to run this notebook.

  develop
* example-patchpredictor
  feature-models


In [5]:
from tiatoolbox.models.classification import patch_predictor
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

mpl.rcParams['figure.dpi'] = 300 # for high resolution figure in notebook

ModuleNotFoundError: No module named 'tiatoolbox.models'

### Downloading the required files
We download, over the internet, a couple of files (a histology image and a csv file containing the positions of nuclei in that image). Download is needed once in each Colab session.

In [ ]:
import requests

img_file_name = "sample_img.png"
csv_file_name = "sample_coordinates.csv"

# Downloading sample image from MoNuSeg
r = requests.get("https://tiatoolbox.dcs.warwick.ac.uk/testdata/patchextraction/TCGA-HE-7130-01Z-00-DX1.png")
with open(img_file_name, "wb") as f:
    f.write(r.content)

# Downloading points list 
r = requests.get("https://tiatoolbox.dcs.warwick.ac.uk/testdata/patchextraction/sample_patch_extraction.csv")
with open(csv_file_name, "wb") as f:
    f.write(r.content)

print('Download is complete.')